In [1]:
import numpy as np
import random
import pandas as pd

import datetime
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dpath = "/content/drive/My Drive/LAB/data/"
files = os.listdir(dpath)
files.sort()
print(files)

['ADANIPORTS.csv', 'ASIANPAINT.csv', 'AXISBANK.csv', 'BAJAJ-AUTO.csv', 'BAJAJFINSV.csv', 'BAJFINANCE.csv', 'BHARTIARTL.csv', 'BPCL.csv', 'BRITANNIA.csv', 'CIPLA.csv', 'COALINDIA.csv', 'DRREDDY.csv', 'EICHERMOT.csv', 'GAIL.csv', 'GRASIM.csv', 'HCLTECH.csv', 'HDFC.csv', 'HDFCBANK.csv', 'HEROMOTOCO.csv', 'HINDALCO.csv', 'HINDUNILVR.csv', 'ICICIBANK.csv', 'INDUSINDBK.csv', 'INFY.csv', 'IOC.csv', 'ITC.csv', 'JSWSTEEL.csv', 'KOTAKBANK.csv', 'LT.csv', 'MARUTI.csv', 'MM.csv', 'NESTLEIND.csv', 'NTPC.csv', 'ONGC.csv', 'POWERGRID.csv', 'RELIANCE.csv', 'SBIN.csv', 'SHREECEM.csv', 'SUNPHARMA.csv', 'TATAMOTORS.csv', 'TATASTEEL.csv', 'TCS.csv', 'TECHM.csv', 'TITAN.csv', 'ULTRACEMCO.csv', 'UPL.csv', 'VEDL.csv', 'WIPRO.csv', 'ZEEL.csv']


In [4]:
df = pd.read_csv(dpath + files[0]).set_index('Date')
df = df[['Close']]

In [5]:
scaler = MinMaxScaler(feature_range=(-1, 1))
df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))

In [6]:
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []

    # create all possible sequences of length look_back
    for index in range(len(data_raw) - look_back):
        data.append(data_raw[index: index + look_back])

    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);

    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]

    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]

    return [x_train, y_train, x_test, y_test]

look_back = 30 # choose sequence length
x_train, y_train, x_test, y_test = load_data(df, look_back)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (2634, 29, 1)
y_train.shape =  (2634, 1)
x_test.shape =  (658, 29, 1)
y_test.shape =  (658, 1)


In [7]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [8]:
input_dim = 1
hidden_dim = 30
num_layers = 5
output_dim = 1

class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :])
        return out

model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
loss_fn = torch.nn.MSELoss()

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

GRU(
  (gru): GRU(1, 30, num_layers=5, batch_first=True)
  (fc): Linear(in_features=30, out_features=1, bias=True)
)
22
torch.Size([90, 1])
torch.Size([90, 30])
torch.Size([90])
torch.Size([90])
torch.Size([90, 30])
torch.Size([90, 30])
torch.Size([90])
torch.Size([90])
torch.Size([90, 30])
torch.Size([90, 30])
torch.Size([90])
torch.Size([90])
torch.Size([90, 30])
torch.Size([90, 30])
torch.Size([90])
torch.Size([90])
torch.Size([90, 30])
torch.Size([90, 30])
torch.Size([90])
torch.Size([90])
torch.Size([1, 30])
torch.Size([1])


In [9]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
files = os.listdir(dpath)
files.sort()
log = dict()
mpath = '/content/drive/My Drive/LAB/output_gru/'
model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
num_epochs = 100
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

rmse = []
maes = []

from tqdm import tqdm, trange

for t in trange(num_epochs):
    for file in files:
        try:
            df = pd.read_csv(dpath + file).set_index('Date')
            df = df[['Close']]
            scaler = MinMaxScaler(feature_range=(-1, 1))
            df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))
            look_back = 30 # choose sequence length
            x_train, y_train, x_test, y_test = load_data(df, look_back)

            x_train = torch.from_numpy(x_train).type(torch.Tensor)
            x_test = torch.from_numpy(x_test).type(torch.Tensor)
            y_train = torch.from_numpy(y_train).type(torch.Tensor)
            y_test = torch.from_numpy(y_test).type(torch.Tensor)

            loss_fn = torch.nn.MSELoss()
            hist = np.zeros(num_epochs)
            seq_dim =look_back-1

            y_train_pred = model(x_train)

            loss = loss_fn(y_train_pred, y_train)
            print(f"{file}: Epoch ", t, "MSE: ", loss.item(), end = '\r')
            hist[t] = loss.item()

            # Zero out gradient, else they will accumulate between epochs
            optimiser.zero_grad()

            # Backward pass
            loss.backward()

            # Update parameters
            optimiser.step()

            y_test_pred = model(x_test)

            # invert predictions
            if t == num_epochs - 1:
                y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
                y_train = scaler.inverse_transform(y_train.detach().numpy())
                y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
                y_test = scaler.inverse_transform(y_test.detach().numpy())
                mean_abs_percentage_error = MeanAbsolutePercentageError()
                log[file] = mean_abs_percentage_error(y_test_pred, y_test)

                mae = mean_absolute_error(y_test, y_test_pred)
                maes.append(mae)
                rmse.append(sqrt(mean_squared_error(y_test, y_test_pred)))

        except:
            log[file] = 'ERROR'

torch.save(model.state_dict(), mpath + 'global_gru' + '.pth')

NameError: name 'os' is not defined

In [10]:
import json
for key, val in log.items():
    log[key] = str(val)
with open(mpath + 'log.json', 'w') as json_file:
    json.dump(log, json_file)

NameError: name 'log' is not defined

# Inference

In [11]:
import numpy as np
import random
import pandas as pd

import datetime
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import matplotlib.pyplot as plt
import os

In [12]:
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []

    # create all possible sequences of length look_back
    for index in range(len(data_raw) - look_back):
        data.append(data_raw[index: index + look_back])

    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);

    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]

    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]

    return [x_train, y_train, x_test, y_test]

In [13]:
def MAPE(Y_Predicted,Y_actual):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [14]:
input_dim = 1
hidden_dim = 30
num_layers = 5
output_dim = 1
model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

import matplotlib.dates as mdates

model.load_state_dict(torch.load('/content/drive/My Drive/LAB/output_gru/global_gru.pth'))
model.eval()

log_hist = ""

def infer(file):
    global log_hist
    dpath = "/content/drive/My Drive/LAB/data/"
    mpath = '/content/drive/My Drive/LAB/output_gru/'
    df = pd.read_csv(dpath + file).set_index('Date')
    df.index = pd.to_datetime(df.index)
    df = df[['Close']]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))
    look_back = 30 # choose sequence length
    x_train, y_train, x_test, y_test = load_data(df, look_back)
    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    y_train = torch.from_numpy(y_train).type(torch.Tensor)
    y_test = torch.from_numpy(y_test).type(torch.Tensor)
#     print(x_train, x_train.shape)
    with torch.no_grad():
        y_train_pred = model(x_train)
        y_test_pred = model(x_test)
    y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
    y_train = scaler.inverse_transform(y_train.detach().numpy())
    y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
    y_test = scaler.inverse_transform(y_test.detach().numpy())

    err_test, err_train = MAPE(y_test_pred, y_test), MAPE(y_train_pred, y_train)

    mae = mean_absolute_error(y_test, y_test_pred)
    maes.append(mae)
    rmse.append(sqrt(mean_squared_error(y_test, y_test_pred)))

    logln = f"{file.split()[0]} had {err_train}% train error and {err_test}% test error\n"
    # print(logln)
    log_hist += logln

    figure, axes = plt.subplots(figsize=(15, 6))
    axes.xaxis.set_major_locator(mdates.AutoDateLocator())
    axes.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    axes.plot(df[len(df)-len(y_test):].index, y_test, color = 'red', label = 'Real Stock Price')
    axes.plot(df[len(df)-len(y_test):].index, y_test_pred, color = 'blue', label = 'Predicted Stock Price')
    #axes.xticks(np.arange(0,394,50))
    plt.title(f'{file} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    # plt.savefig('/content/drive/My Drive/LAB/output_gru/' + file.split('.')[0] + '.png')
    # plt.show()

<ipython-input-14-4c6399614b7e>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/My Drive/LAB/output_gru/global_gru.pth'))


In [17]:
rmse = []
maes = []

from tqdm import tqdm
for file in tqdm(files):
    try:
        infer(file)
    except:
        continue

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
log_hist

'ADANIPORTS.csv had 3.1433526426553726% train error and 1.9431313499808311% test error\nASIANPAINT.csv had 23.12905341386795% train error and 1.4791152440011501% test error\nAXISBANK.csv had 14.222632348537445% train error and 1.8538303673267365% test error\nBAJAJ-AUTO.csv had 2.3248959332704544% train error and 1.4736691489815712% test error\nBAJAJFINSV.csv had 9.644678235054016% train error and 1.9680676981806755% test error\nBAJFINANCE.csv had 94.95505094528198% train error and 2.2554857656359673% test error\nBHARTIARTL.csv had 3.7963062524795532% train error and 1.8069565296173096% test error\nBPCL.csv had 5.027914419770241% train error and 1.938488893210888% test error\nBRITANNIA.csv had 15.780319273471832% train error and 1.3836869969964027% test error\nCIPLA.csv had 3.440370410680771% train error and 1.4003721065819263% test error\nCOALINDIA.csv had 1.3557816855609417% train error and 1.9177470356225967% test error\nDRREDDY.csv had 2.9042595997452736% train error and 1.560452673

In [ ]:
with open('/content/drive/My Drive/LAB/output_gru/LOG.log', 'w+') as f:
    f.write(log_hist)
f.close()

In [ ]:

y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
                y_train = scaler.inverse_transform(y_train.detach().numpy())
                y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
                y_test = scaler.inverse_transform(y_test.detach().numpy())
                mean_abs_percentage_error = MeanAbsolutePercentageError()
                log[file] = mean_abs_percentage_error(y_test_pred, y_test)



In [18]:
print(rmse)
print(maes)

[54.49008564494783, 464.0576641162173, 70.59647861592497, 118.34042855782634, 195.73331665815098, 435.29702503003625, 21.190591502194803, 127.25923959579517, 793.8368062769576, 425.48621085177365, 4.063355810548128, 65.73438428511938, 530.1592036831955, 16.89824877813018, 105.50787237364092, 94.08715935071852, 401.165080577809, 319.43702920137486, 300.6406051043338, 53.85636505224452, 322.9206800825862, 52.2784690869781, 32.653294947040756, 1430.3862677612644, 77.2704671763815, 127.54395305516252, 3.66553030322621, 582.6669556873807, 109.54037943408129, 152.8976373264152, 112.91180705256204, 283.39697201981534, 2.509373069372895, 15.22987263196059, 3.4740381570059085, 238.41616877059744, 68.76291427002094, 404.21397489448583, 1335.4254939905857, 35.147828852531916, 19.483589505267393, 340.34490777738984, 31.11345363456639, 231.5815950291603, 91.57201315767826, 188.47553513321031, 107.14969267629749, 1876.454302667667, 20.495641477642952]
[35.951004, 311.2912, 49.879814, 81.3101, 129.33